In [ ]:
import os
from unittest.mock import patch

from op_analytics.coreutils.partitioned import dailydata
from op_analytics.coreutils.partitioned.location import DataLocation


def mock_location():
    return DataLocation.GCS

os.environ["ALLOW_WRITE"] = "true"

with patch.object(dailydata, "determine_location", mock_location):
    
    # NOTE: Before running these the LAST_N_DAYS values were manually modified.
    
    # Protocols (modified to 120 days)
    from op_analytics.datasources.defillama import protocols
    result = protocols.execute_pull()
    
    # Stablecoins (modified to 120 days)
    # from op_analytics.datasources.defillama import stablecoins
    # result = stablecoins.execute_pull()
    
    # Historical chain TVL (modified to 120 days)
    # from op_analytics.datasources.defillama import historical_chain_tvl
    # result = historical_chain_tvl.execute_pull()
    
    # Volume, Fees, Revenue (modified to 120 days)
    # from op_analytics.datasources.defillama.volume_fees_revenue import execute
    # result = execute.execute_pull()
    
    # TVL breakdown (modifield to 120 days)
    # from op_analytics.datasources.defillama import tvl_breakdown_enrichment
    # result = tvl_breakdown_enrichment.execute_pull()